## 本物のコード

In [156]:
# 無限ループの中で任意秒待つためのモジュール
import time
# MySQLに接続するためのモジュール
import mysql.connector
# ウェブサイトからHTMLを取ってくるためのモジュール
import urllib
# 曲名で検索した結果を表示してる画面のURLのもと(URLtoSearchByTitle[1] には後から曲の名前が入る)
listURLtoSearchByTitle = ["https://www.uta-net.com/search/?Aselect=2&Keyword=", "", "&Bselect=4"]
# HTMLパーサー
from bs4 import BeautifulSoup


while True:
    print("check if the programming is running")
    time.sleep(2) #2秒待ってから実行
    
    # DBに接続する
    connection = mysql.connector.connect(user='root', host='localhost', database='lyrics_database2')
    # カーソルを取得する
    cursor = connection.cursor()
    # クエリの実行
    cursor.execute(
        "SELECT singer.id, singer.name, song.title, song.lyric " +
        "FROM song " +
        "INNER JOIN singer_song ON singer_song.song_id = song.id " +
        "INNER JOIN composer ON composer.id = song.composer_id " +
        "INNER JOIN singer ON singer.id = singer_song.singer_id;"
    )
    # 実行結果をすべて取得
    rows = cursor.fetchall()
    
    for row in rows:
        lyrics = row[3]
        # 歌詞がNULLの時は歌詞をweb(Uta-Net)に取りに行く
        if lyrics is None:
            ###### まずは曲名で検索 ######
            singerName = row[2]
            # 配列 listURLtoSearchByTitleを成型して、文字列stringURLtoSearchByTitleにする
            listURLtoSearchByTitle[1] = singerName
            stringURLtoSearchByTitle = "".join(listURLtoSearchByTitle)
            # HTMLファイルを開く
            data = urllib.request.urlopen(stringURLtoSearchByTitle)
            # HTMLの取得
            html = data.read()
            # HTMLファイルを閉じる
            data.close()
            # BeautifulSoupオブジェクトの作成
            soup = BeautifulSoup(html)
            # 取得したいタグの種類とクラス属性を指定し、歌手名をリストで取得
            singerNames = soup.findAll("td", class_="td2")
            
            
            ### 曲名を元に検索して出てきたものの中から歌手名も一致するものを選択する ###
            # for文の最初で+1するため、indicatorの初期値は0ではなく-1にしておく
            indicator = -1
            # targetの定義
            target = ""
            
            
            ## 検索したい曲のページのURLの取得
            for td in singerNames:
                indicator += 1
                targetSingerName = td.find("a").text
                if targetSingerName == singerName:
                    urls = soup.findAll("td", class_ = "side td1")
                    kk = -1
                    for td2 in urls:
                        kk += 1
                        targetURL = td2.find("a").attrs['href']
                        if kk == indicator:
                            target = "https://www.uta-net.com" + targetURL
                            break
                    break
                #else: # 曲名で検索した結果がなかった時
                    
                    
            ## 検索したいページを開いてから歌詞を手に入れるまでのながれ
            data = urllib.request.urlopen(target)
            # HTMLの取得
            html = data.read()
            # HTMLファイルを閉じる
            data.close()
            # BeautifulSoupオブジェクトの作成
            soup = BeautifulSoup(html, "lxml")
            # 取得したいタグの種類とクラス属性を指定し、歌詞を取得
            lyricsFromHTML = soup.find("div", id = "kashi_area")
            # lyricsFromHTMLを成型(文字列にしてからhtmlタグを削除)
            stringLyricsFromHTML = str(lyricsFromHTML)
            parsedLyrics = stringLyricsFromHTML.replace('<div id="kashi_area">', "").replace("</div>", "").replace("<br/>", "")
            
            
            # DBに歌詞を書き込み##################################
            cursor.execute(
                #update文に変える
                "UPDATE lyrics_database.song SET lyric = '" + parsedLyrics + "'" +
                "WHERE id = " + int(row[0]) + ";"
            )
            connection.commit()
            
    cursor.close
    connection.close

check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running
check if the programming is running


KeyboardInterrupt: 

## コードがちゃんと動くかを、DBに接続しないでテスト中

In [153]:
import urllib
# 曲名で検索した結果を表示してる画面のURLのもと(URLtoSearchByTitle[1] には後から曲の名前が入る)
# HTMLパーサー
from bs4 import BeautifulSoup


data = urllib.request.urlopen("https://www.uta-net.com/search/?Aselect=2&Keyword=%E8%99%B9&Bselect=4")
# HTMLの取得
html = data.read()
# HTMLファイルを閉じる
data.close()
# BeautifulSoupオブジェクトの作成
soup = BeautifulSoup(html, "lxml")
# 取得したいタグの種類とクラス属性を指定し、歌手名をリストで取得
singerNames = soup.findAll("td", class_ = "td2")
# for文の最初に+1するため、indicatorの初期値は0ではなく-1にしておく
indicator = -1
# targetの定義
target = ""

## 検索したい曲のページのURLの取得
for td in singerNames:
    indicator += 1
    targetSingerName = td.find("a").text
    if targetSingerName == "Aqua Timez":
        urls = soup.findAll("td", class_ = "side td1")
        kk = -1
        for td2 in urls:
            kk += 1
            targetURL = td2.find("a").attrs['href']
            if kk == indicator:
                target = "https://www.uta-net.com" + targetURL
                break
        break


## 検索したいページを開いてから歌詞を手に入れるまでのながれ
data = urllib.request.urlopen(target)
# HTMLの取得
html = data.read()
# HTMLファイルを閉じる
data.close()
# BeautifulSoupオブジェクトの作成
soup = BeautifulSoup(html, "lxml")
# 取得したいタグの種類とクラス属性を指定し、歌詞を取得
lyricsFromHTML = soup.find("div", id = "kashi_area")
# lyricsFromHTMLを成型(文字列にしてからhtmlタグを削除)
stringLyricsFromHTML = str(lyricsFromHTML)
parsedLyrics = stringLyricsFromHTML.replace('<div id="kashi_area">', "").replace("</div>", "").replace("<br/>", "")


print(parsedLyrics)

大丈夫だよ　見上げれば　もう大丈夫ほら　七色の橋やっと同じ空の下で　笑えるね靴紐を結びなおす時　風が僕らの背中を押す空がこぼした光の向こうに　あのユメの続きを描こう左胸の奥が高鳴る　期待と不安が脈を打つ本当に大丈夫かな　全て乗り越えてゆけるかな大丈夫だよ　見上げれば　もう大丈夫ほら　七色の橋涙を流しきると　空に架かるねぇ見えるでしょ　はるか彼方に僕にも見える　君と同じの二つの空が　いま一つになるやっと同じ空の下で　笑えるね別々の空を持って生まれた　記憶を映し出す空君には君の物語があり　僕の知らない涙があるもしかしたら僕が笑う頃に　君は泣いてたのかもしれない似たような喜びはあるけれど　同じ悲しみはきっとない「約束」で未来を縁取り　コトバで飾り付けをする君は確かな明日を　きっと　誰より　欲しがってた巡る季節のひとつのように悲しい時は　悲しいままに幸せになることを　急がないで大丈夫だよ　ここにいるから大丈夫だよ　どこにもいかないまた走り出す時は　君といっしょ「涙のない世界にも　その橋は架かりますか？」壁に刻まれた落書きは　ダレカの字によく似てた悲しみを遠ざけることで　君は橋を架けようとしたけれど　今　傘を捨てて　目をつぶるだいじょうぶ大丈夫だよ　見上げれば　もう大丈夫ほら　七色の橋涙を流し終えた君の空にねぇ見えるでしょ　色鮮やかに僕にも見える　君と同じの絆という名の虹が架かったねそして二つの空がやっと　やっと　一つになって僕らを走らせるんだ
